# ユニバーサル Abacus マシン

## 2分探索

自然数上の単調関数 $f$ を計算するプログラムに対し，逆関数を求めるプログラム。2分探索で
行う。

In [4]:
# 練習問題40

def bsearch (p, f, b):
    "For monotonic f, binary search least n such that p <= f(n)  up to b"
    if (f(b) < p):
        return b   # this means that no such n exists
    return bsearchrange (p, f, 0, b)

def bsearchrange (p, f, a, b):
    if(b <= a+1):
        return b
    c = (a+b)//2
    if (f(c) < p):
        return (bsearchrange (p, f, c, b))
    else:
        return (bsearchrange (p, f, a, c))
    
bsearch (36, lambda n:n*n, 100)    


6

## ペアのコーディング

**練習問題50** ここでは，0 以上の数を自然数ということにする。自然数全体の集合$\mathbb N$ と，そのペア全体の集合$\mathbb N \times \mathbb N$ とではどちらの方が要素が多いだろうか？もちろん両方とも無限個存在するが、ペア全体に
三角形に番号をつける，つまり (0,0), (1,0), (0,1), (2.0), (1,1), (0,2), (3.0), ... と番号をつけていくことにより，$\mathbb N \times \mathbb N$ と $\mathbb N$ は1対1対応をつけることができる(カントールの対関数)。
この1対1対応を作る $\mathbb N \times \mathbb N \to\mathbb N$の関数 pairnum(x, y) と，その逆関数 numpair(z)
$\mathbb N \to\mathbb N \times\mathbb N$ を作ろう。numpair は，タプルを返せばよい。bsearch を有効に使おう。

逆になっていることを、z=1 から z=100 までnumpair して pairnum して元に戻ることを確認しよう。numpair の結果を pairnum に渡すところで、* を用いる。

In [5]:
#練習問題50 
import math
def pairnum(x, y):
    return (x + y)* (x + y + 1)//2 + y

def numpair(z):
    kk = bsearch(z, lambda i:pairnum(i, 0)-1, z+1) - 1

    r = pairnum(kk, 0)
    y = z - r
    x = (kk) - y
    return (x, y)


# [pairnum(*numpair(i))==i for i in range(100)]


## リストのコーディング

**練習問題60** 自然数の集合$\mathbb N$ と，自然数の有限列(すなわちリスト)全体の集合$\mathbb N^*$ とではどちらの方が大きいだろうか？もちろん有限列全体だろうと思うかもしれない。しかし，次のようにして，
$\mathbb N^*$ と $\mathbb N$ は1対1対応をつけることができる。
$$
\text{listnum}(u) = \left\{\begin{array}{ll} 0 & (u = []) \\ \text{pairnum}(u[0], \text{listnum}(u[1:])+1 & (それ以外)\end{array}\right.
$$
この対応とその逆対応( numlist(z) ) のプログラムを書こう。

このような1対1対応が存在する2つの集合のことを，濃度が等しいという。実数全体の集合や $\mathbb N$ の無限列全体の集合は，$\mathbb N$ と濃度が等しくないことが知られている（その証明は，カントールの対角線論法という手法を使う）。


In [6]:
# 練習問題60

def listnum(u):  #u:list
    if (u==[]):
        return 0
    else:
        return pairnum(u[0], listnum(u[1:]))+ 1

def numlist(n):
    if(n == 0):
        return []
    else:
        (a, b) = numpair(n - 1)
        return([a] + numlist(b))

    
#k = listnum([1,2,3])
#k = listnum([1,1,1,1,1,1,1,1,1,1,1,1])
#print(k)
#print(numlist(k))


In [25]:
def nth(x, k):
    if(x == 0):
        return 0  # 本来なら，エラー
    (a, y) = numpair(x-1)
    if(k == 0):
        return a
    else: 
        return nth(y, k-1)

def replace(x, k, n):   # x[k] = n
    if(x == 0):
        return 0   #本来なら，エラー
    (a, y) = numpair(x-1)
    if(k == 0):
        return pairnum(n, y)+ 1
    else:
        return pairnum(a, replace(y, k-1, n))+ 1

z = listnum([1,2,3,4,5])    
nth(z, 5)
u = replace(z, 4, 8)     
numlist(u)
        
        

[1, 2, 3, 4, 8]

ところで、このプログラムでは、ここら辺が限界で、もっと大きなリストをエンコードしようとしても、bsearchのところでの再帰呼び出しが深くなりすぎて、エラーを生じてしまう。
numpair に関しては、次が知られている。
```
# (wikipedia 「対関数」参照)
import math
def numpair(z):
    w = (int)((math.sqrt(8*z+1)-1)//2)
    t = (w*w+w)//2
    y = z-t
    x = w-y
    return(x,y)
```
これで行えば2分探索の必要はなくなるが、今度は、math.sqrt が, 浮動小数点で扱えるより大きな引数が与えられてエラーをいう。(その前に、大きな数を浮動小数点にした時に誤差が生じる。）それを防ぐには、整数範囲内でのルート計算を行わないといけない。それは、ニュートン法でできる。



# ユニバーサル Abacus マシン

プリントを参照。プリントでは p, d として自然数を与えているが，ここでは python のリストで与えている。また，マシンコードは，リストの長さをそろえて，
```
[b + n]   --> [1, b, n, 0]
[b - n, ne]  --> [2, b, n, ne]
[end]     --> [0,0,0,0]
```
で表現することとする。まだ，データ d は，プログラム中に現れる箱の番号を全て含むように，十分長く 0 を後ろにつけたものを用いることにする。


In [30]:
def univ(p, d):
    pc = 0
    while(p[pc][0] > 0):
        op = p[pc][0]
        box = p[pc][1]
        next = p[pc][2]
        if(op == 1):
            d[box] += 1
            pc = next
        else:
            enext = p[pc][3]
            if(d[box] > 0):
                d[box] -= 1
                pc = next
            else:
                pc = enext
    return(d[0])

# 様々な関数の Abacus マシンでのコード

何もしないでいきなり終了する. どの k に対しても f(x1,...,xk) = 0 を意味する。


In [31]:
m = [[0]]
d = [0,1,6,4,3]   # 引数としては，任意のリスト。d[0] は返値，d[1,...,k] が引数
univ(m,d)

0

箱1 の内容を箱0 にコピーする。箱0は空にする。 f (x) = y を意味する。

In [33]:
d = [0, 5]
m = [[2,1,1,2],
 [1,0,0],
 [0]
]  
univ(m, d)

5

箱 0 の内容を B[1] + B[2] にする。箱 1, 2 は終了時に空になる。
f(x, y) = x+y を意味する。

![足し算](plus_machine.jpg)

In [34]:
d = [0, 5, 3]
m = [[2,1,1,2],
 [1,0,0],
 [2,2,3,4],
 [1,0,2],
 [0]
]  
univ(m, d)

8

''''

mm : このプログラムの，コード（という数）
dd : このデータのリストを数として表現したもの

In [41]:
m0 = list (map(listnum, m))
print(m0)
mm = listnum (m0)
print(mm)
dd = listnum(d)
print(dd)

[32638, 14, 26364689, 152, 1]
1830392212574384253594576266619768224916214664365824640643
70


In [45]:
def univm(p, d):  
    ''' p, d は，リストを数で表した数が与えられる '''

    pc = 0
    while(nth(nth(p, pc), 0) > 0):
        op = nth(nth(p, pc), 0)
        box = nth(nth(p, pc), 1)
        next = nth(nth(p, pc), 2)
        if(op == 1):
            replace(d, box, nth(d, box)+1)
            pc = next
        else:
            enext = nth(nth(p, pc), 3)
            if(nth(d,box) > 0):
                if(nth(d, box) > 0):
                    replace(d, box, nth(d, box) - 1)
                pc = next
            else:
                pc = enext
    return(nth(d,0))

In [46]:
univm(mm, dd)

8

**練習問題** この下に，次のセルを作成して，引き算のプログラムに対して同様のことを行おう。

1. 引き算のプログラムを，Abacus マシンで作成しよう。(つまり，マシンが停止した時
に，B[0] の値が B[1] - B[2] となっているマシン。B[1], B[2] の値は変化していてもよい）できれば，その絵を .jpg ファイルなどにして，この notebook に取り込もう。

2. それをリストでの表現になおし，listnum を用いて，コンパイルしたマシンコードとなる数を求めよう。

3. 適当な引数 (上と同じ，5, 3 でもよい)　に対して univm で動作させ，結果が正しいことを確認しよう。